In [3]:
def read_first(read=False):
    print("set your ngrok token first, and set your 'env.yaml' file.")
    if not read:
        raise "This is just a break point. Just remove this line after completing the tasks"

In [4]:
read_first()

set your ngrok token first, and set your 'env.yaml' file.


TypeError: exceptions must derive from BaseException

In [1]:
import os

!chmod +x install.sh
!./install.sh

ollama command not found. Installing ollama...
>>> Downloading ollama...
######################################################################## 100.0%#=#=#                                                                                  13.2%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Miniconda3 directory already exists. Skipping installation.


In [2]:
from interface import set_environment, start, set_server, load_model, check_servers, shutdown_servers

In [3]:
model_name = "llama2:13b"

yaml_content = f"""
model_list:
  - model_name: gpt-4
    litellm_params:
      model: ollama/{model_name}

  - model_name: gpt-3.5-turbo
    litellm_params:
      model: ollama/{model_name}

litellm_settings:
  callbacks: custom_callbacks.proxy_handler_instance
  drop_params: True
"""

file_path = "litellm/config.yaml"

with open(file_path, "w") as file:
    file.write(yaml_content)

print(f"{file_path} file was generated.")

litellm/config.yaml file was generated.


In [4]:
set_environment()

Environment variables have been set.


In [5]:
start()

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libutempter0
Suggested packages:
  byobu | screenie | iselect ncurses-term
The following NEW packages will be installed:
  libutempter0 screen
0 upgraded, 2 newly installed, 0 to remove and 3 not upgraded.
Need to get 586 kB of archives.
After this operation, 1073 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libutempter0 amd64 1.1.6-4 [8256 B]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 screen amd64 4.8.0-1ubuntu0.1 [577 kB]


dpkg-preconfigure: unable to re-open stdin: No such file or directory


Fetched 586 kB in 0s (5071 kB/s)
Selecting previously unselected package libutempter0:amd64.
(Reading database ... 69943 files and directories currently installed.)
Preparing to unpack .../libutempter0_1.1.6-4_amd64.deb ...
Unpacking libutempter0:amd64 (1.1.6-4) ...
Selecting previously unselected package screen.
Preparing to unpack .../screen_4.8.0-1ubuntu0.1_amd64.deb ...
Unpacking screen (4.8.0-1ubuntu0.1) ...
Setting up libutempter0:amd64 (1.1.6-4) ...
Setting up screen (4.8.0-1ubuntu0.1) ...
Processing triggers for man-db (2.9.1-1) ...
Processing triggers for install-info (6.7.0.dfsg.2-5) ...
Processing triggers for libc-bin (2.31-0ubuntu9.9) ...
Processing triggers for systemd (245.4-4ubuntu3.19) ...
no change     /notebooks/oneclick/miniconda3/condabin/conda
no change     /notebooks/oneclick/miniconda3/bin/conda
no change     /notebooks/oneclick/miniconda3/bin/conda-env
no change     /notebooks/oneclick/miniconda3/bin/activate
no change     /notebooks/oneclick/miniconda3/bin/dea

In [6]:
set_server(phase=1)

In [7]:
import time
import subprocess

# Give some time to the server
time.sleep(5)

# load a model
command = f"ollama run {model_name} hi"
subprocess.run(command, shell=True)

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest 
pulling 2609048d349e...   0% ▕                ▏    0 B/7.4 GB                  pulling manifest 
pulling 2609048d349e...   0% ▕                ▏    0 B/7.4 GB                  pulling manifest 
pulling 2609048d349e...   0% ▕                ▏    0 B/7.4 GB                  pulling manifest 
pulling 2609048d349e...   0% ▕                ▏    0 B/7.4 GB                  pulling manifest 
pulling 2609048d349e...   0% ▕                ▏    0 B/7.4 GB                  pulling manifest 
pulling 2609048d349e...   0% ▕                ▏    0 B/7.4 GB                  pulling manifest 
pulling 2609048d349e...   0% ▕                ▏    0 B/7.4 GB                  pulling manifest 
pulling 2609048d349e...   0% ▕       


Hello! How can I help you today?



CompletedProcess(args='ollama run llama2:13b hi', returncode=0)

In [8]:
# Use the above shell. This doesn't generate the output to monitor.

# load_model(model_name = model_name)

In [10]:
set_server(phase=2)
set_server(phase=3)

In [11]:
# Give some time to the server
time.sleep(5)

In [12]:
check_servers()

You must see 3 Socktes
There are screens on:
	575.remote_serve	(03/03/24 17:55:54)	(Detached)
	565.litellm_serve	(03/03/24 17:55:54)	(Detached)
	466.ollama	(03/03/24 17:54:55)	(Detached)
3 Sockets in /run/screen/S-root.

Your endpoint is : https://0a2b-172-83-13-4.ngrok-free.app

models available in your endpoint:
{"data":[{"model_name":"gpt-4","litellm_params":{"model":"ollama/llama2:13b"},"model_info":{"max_tokens":4096,"input_cost_per_token":0.0,"output_cost_per_token":0.0,"litellm_provider":"ollama","mode":"completion"}},{"model_name":"gpt-3.5-turbo","litellm_params":{"model":"ollama/llama2:13b"},"model_info":{"max_tokens":4096,"input_cost_per_token":0.0,"output_cost_per_token":0.0,"litellm_provider":"ollama","mode":"completion"}}]}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   430  100   430    0     0   209k      0 --:--:-- --:--:-- --:--:--  209k


In [ ]:
with open("server_info.txt", "r") as file:
    url = file.read()
url

In [ ]:
### remote server test
import requests

data = {
    "model":"gpt-4",
    "messages":[{ "content": "Hi","role": "user"}],
}

headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer OPENAI_API_KEY'
}

response = requests.post(
    f"{url}/chat/completions",
    headers=headers,
    json=data,
    stream=True,
)

#response.json()["choices"][0]["message"]
response.json()

If there is a problem, try below.

- run shutdown_servers()
- rerun set_server(1), (2), (3)
- or, just run reset_servers()

In [ ]:
# shutdown_servers()

In [ ]:
def reset_servers():
    shutdown_servers()
    set_server(1)
    set_server(2)
    set_server(3)
    check_servers()